# 🔍 Diagnostic et Correction de la Recherche Vectorielle RAG

Ce notebook diagnostique et corrige le problème de recherche vectorielle dans le système RAG qui empêche de trouver "Mondial Tissus".

## Problème identifié :
- ✅ Recherche textuelle simple fonctionne : trouve "mondial tissus" 
- ❌ Recherche vectorielle échoue : `'str' object has no attribute 'get'`

## Objectif :
Corriger le bug de recherche vectorielle pour que le Chat RAG puisse trouver "Mondial Tissus"

In [1]:
# Import Required Libraries
import sys
import os
import pickle
import pandas as pd
from pathlib import Path

# Ajouter le répertoire rag_app au path (version Jupyter)
current_dir = os.getcwd()
sys.path.insert(0, current_dir)

try:
    from rag_app.core.vector_database import VectorDatabase
    print("✅ Imports RAG réussis")
except ImportError as e:
    print(f"❌ Erreur d'import RAG: {e}")
    print(f"📁 Répertoire actuel: {current_dir}")
    print(f"📋 Contenu du répertoire:")
    for item in os.listdir(current_dir):
        print(f"   {item}")

import warnings
warnings.filterwarnings('ignore')

✅ Imports RAG réussis


In [2]:
# Charger la base vectorielle
db_path = 'data/docs/vector_db.pkl'

if os.path.exists(db_path):
    with open(db_path, 'rb') as f:
        db = pickle.load(f)
    
    print(f"📊 Base vectorielle chargée : {len(db.documents)} documents")
    
    # Examiner la structure des documents
    if db.documents:
        sample_doc = db.documents[0]
        print(f"📋 Structure du premier document:")
        print(f"   Type: {type(sample_doc)}")
        print(f"   Clés: {list(sample_doc.keys()) if isinstance(sample_doc, dict) else 'Non-dict'}")
        
        # Vérifier les métadonnées
        if isinstance(sample_doc, dict) and 'metadata' in sample_doc:
            metadata = sample_doc['metadata']
            print(f"   Métadonnées type: {type(metadata)}")
            if isinstance(metadata, dict):
                print(f"   Clés métadonnées: {list(metadata.keys())}")
            
else:
    print("❌ Base vectorielle non trouvée")

📊 Base vectorielle chargée : 498 documents
📋 Structure du premier document:
   Type: <class 'dict'>
   Clés: ['text', 'metadata', 'timestamp', 'type']
   Métadonnées type: <class 'dict'>
   Clés métadonnées: ['source', 'title', 'category', 'project', 'author', 'date', 'description', 'tags', 'priority', 'status']


In [3]:
# Test de recherche textuelle simple
def test_simple_text_search(query, documents):
    """Recherche textuelle simple dans les documents."""
    results = []
    
    for i, doc in enumerate(documents):
        if isinstance(doc, dict):
            text = doc.get('text', '').lower()
            metadata = doc.get('metadata', {})
            source = metadata.get('source', f'Document_{i}') if isinstance(metadata, dict) else f'Document_{i}'
        else:
            text = str(doc).lower()
            source = f'Document_{i}'
        
        if query.lower() in text:
            results.append((i, source, text[:100] + "..."))
    
    return results

# Test avec "mondial tissus"
query = "mondial tissus"
text_results = test_simple_text_search(query, db.documents)

print(f"🔍 Recherche textuelle '{query}': {len(text_results)} résultats")
for i, (idx, source, excerpt) in enumerate(text_results):
    print(f"  📄 {idx}: {os.path.basename(source)}")
    print(f"      Extrait: {excerpt}")
    print()

🔍 Recherche textuelle 'mondial tissus': 2 résultats
  📄 21: M401_annonce_.pdf
      Extrait: • développeur.euse confirmé.e  
groupe mondial tissus  
3.33.3 étoile(s) sur 5  
69140 rillieux -la-...

  📄 25: M401_LM_MondialTissus_DevConfirmé.pdf
      Extrait: cyril sauret  
60, route de feurs  
69170 tarare  
 
07.86.09.87.81  
sauretcyril@outlook.fr  
 
   ...



In [4]:
# Test de recherche vectorielle avec debug
def debug_vector_search(vector_db, query, top_k=5):
    """Test la recherche vectorielle avec diagnostic détaillé."""
    print(f"🔍 Debug recherche vectorielle pour: '{query}'")
    
    try:
        # Vérifier l'état de la base vectorielle
        print(f"📊 Documents dans la base: {len(vector_db.documents)}")
        print(f"📊 Vecteurs initialisés: {vector_db.vectors is not None}")
        
        if vector_db.vectorizer is None:
            print("❌ Vectorizer non initialisé")
            return []
        
        # Essayer la recherche
        results = vector_db.search(query, top_k=top_k)
        print(f"✅ Recherche réussie: {len(results)} résultats")
        
        # Examiner les résultats
        for i, result in enumerate(results):
            print(f"📄 Résultat {i+1}:")
            print(f"   Type: {type(result)}")
            
            if isinstance(result, dict):
                print(f"   Clés: {list(result.keys())}")
                metadata = result.get('metadata', {})
                if isinstance(metadata, dict):
                    source = metadata.get('source', 'N/A')
                    print(f"   Source: {os.path.basename(source)}")
                else:
                    print(f"   Métadonnées: {metadata} (type: {type(metadata)})")
            else:
                print(f"   Contenu: {str(result)[:100]}...")
        
        return results
        
    except Exception as e:
        print(f"❌ Erreur lors de la recherche vectorielle: {e}")
        import traceback
        traceback.print_exc()
        return []

# Test de la recherche vectorielle
print("=" * 60)
vector_results = debug_vector_search(db, "COSERVICES")

🔍 Debug recherche vectorielle pour: 'COSERVICES'
📊 Documents dans la base: 498
📊 Vecteurs initialisés: True
✅ Recherche réussie: 3 résultats
📄 Résultat 1:
   Type: <class 'dict'>
   Clés: ['document', 'similarity']
   Source: A
📄 Résultat 2:
   Type: <class 'dict'>
   Clés: ['document', 'similarity']
   Source: A
📄 Résultat 3:
   Type: <class 'dict'>
   Clés: ['document', 'similarity']
   Source: A


In [6]:
# Debug spécifique de l'erreur 'str' object has no attribute 'get'
def fix_vector_search_bug(vector_db):
    """Analyse et corrige le bug de recherche vectorielle."""
    print("🔧 DIAGNOSTIC DU BUG 'str' object has no attribute 'get'")
    print("=" * 60)
    
    # Examiner les documents pour identifier le problème
    problematic_docs = []
    
    for i, doc in enumerate(vector_db.documents):
        if not isinstance(doc, dict):
            problematic_docs.append((i, type(doc), str(doc)[:100]))
        elif 'metadata' in doc:
            metadata = doc['metadata']
            if not isinstance(metadata, dict):
                problematic_docs.append((i, f"metadata is {type(metadata)}", str(metadata)[:100]))
    
    if problematic_docs:
        print(f"❌ Documents problématiques trouvés: {len(problematic_docs)}")
        for i, (idx, issue, content) in enumerate(problematic_docs[:3]):
            print(f"   {idx}: {issue}")
            print(f"       Contenu: {content}...")
    else:
        print("✅ Tous les documents semblent avoir une structure correcte")
    
    # Essayer de corriger la structure si nécessaire
    if problematic_docs:
        print("\n🔧 Tentative de correction...")
        corrected_docs = []
        
        for doc in vector_db.documents:
            if isinstance(doc, dict):
                # S'assurer que metadata est un dict
                if 'metadata' in doc and not isinstance(doc['metadata'], dict):
                    doc['metadata'] = {'source': str(doc['metadata'])}
                corrected_docs.append(doc)
            else:
                # Convertir les non-dict en dict
                corrected_docs.append({
                    'text': str(doc),
                    'metadata': {'source': f'converted_doc_{len(corrected_docs)}'},
                    'timestamp': '',
                    'type': 'converted'
                })
        
        vector_db.documents = corrected_docs
        print(f"✅ Structure corrigée: {len(corrected_docs)} documents")
        return True
    
    return False

# Appliquer le fix
fix_applied = fix_vector_search_bug(db)

🔧 DIAGNOSTIC DU BUG 'str' object has no attribute 'get'
✅ Tous les documents semblent avoir une structure correcte


In [7]:
# Test après correction
if fix_applied:
    print("\n🧪 TEST APRÈS CORRECTION")
    print("=" * 40)
    
    # Re-tester la recherche vectorielle
    vector_results_fixed = debug_vector_search(db, "mondial tissus")
    
    if vector_results_fixed:
        print("\n🎉 SUCCÈS! La recherche vectorielle fonctionne maintenant!")
        
        # Sauvegarder la base corrigée
        try:
            with open(db_path, 'wb') as f:
                pickle.dump(db, f)
            print("💾 Base vectorielle corrigée sauvegardée")
        except Exception as e:
            print(f"❌ Erreur sauvegarde: {e}")
    else:
        print("❌ La correction n'a pas résolu le problème")
        
else:
    print("\n✅ Aucune correction nécessaire")
    print("Le problème de recherche vectorielle pourrait être ailleurs")


✅ Aucune correction nécessaire
Le problème de recherche vectorielle pourrait être ailleurs


In [ ]:
# Comparaison finale des résultats
print("\n📊 COMPARAISON RECHERCHE TEXTUELLE vs VECTORIELLE")
print("=" * 60)

# Récapitulatif des résultats textuels
print(f"🔤 Recherche textuelle:")
print(f"   Résultats: {len(text_results)}")
for i, (idx, source, excerpt) in enumerate(text_results):
    print(f"   📄 {os.path.basename(source)}")

# Récapitulatif des résultats vectoriels
print(f"\n🔍 Recherche vectorielle:")
if 'vector_results_fixed' in locals() and vector_results_fixed:
    print(f"   Résultats: {len(vector_results_fixed)}")
    for i, result in enumerate(vector_results_fixed):
        if isinstance(result, dict) and 'metadata' in result:
            source = result['metadata'].get('source', 'N/A')
            print(f"   📄 {os.path.basename(source)}")
        else:
            print(f"   📄 Résultat {i+1}")
else:
    print("   ❌ Échec ou pas de correction appliquée")

print(f"\n🎯 CONCLUSION:")
if len(text_results) > 0:
    print("✅ Les documents contenant 'Mondial Tissus' existent dans la base")
    if 'vector_results_fixed' in locals() and vector_results_fixed:
        print("✅ La recherche vectorielle fonctionne après correction")
        print("🚀 Le Chat RAG devrait maintenant trouver 'Mondial Tissus'")
    else:
        print("❌ La recherche vectorielle a encore des problèmes")
        print("🔧 Une investigation plus poussée est nécessaire")
else:
    print("❌ Aucun document contenant 'Mondial Tissus' trouvé")
    print("🔧 Le problème est dans l'indexation des documents")


📊 COMPARAISON RECHERCHE TEXTUELLE vs VECTORIELLE
🔤 Recherche textuelle:
   Résultats: 2
   📄 M401_annonce_.pdf
   📄 M401_LM_MondialTissus_DevConfirmé.pdf

🔍 Recherche vectorielle:
   ❌ Échec ou pas de correction appliquée

🎯 CONCLUSION:
✅ Les documents contenant 'Mondial Tissus' existent dans la base
❌ La recherche vectorielle a encore des problèmes
🔧 Une investigation plus poussée est nécessaire


: 